In [ ]:
Packages <- c("plm", "arrow", "panelvar", "dplyr", "psych", "tidyverse", "lmerTest")
invisible(lapply(Packages, library, character.only = TRUE))

In [ ]:
loincs_cofounder_potassium <- c('2157-6', #creatine kinase
                              'LG6033-7', #AST      
                              '2532-0', #LDH
                              'LG44861-9', #Haptoglobin 
                              'LG49755-8') #uric acid
bp_cofounder_potassium <- c('DIASTOLIC','SYSTOLIC')
loincs_potassium = c('LG49936-4', 'LG6657-3')

In [ ]:
loinc1 <- 'LG49936-4'
loinc2 <- 'LG6657-3'
datafolder <- '/home/hoyinchan/blue/Data/data2022/'

In [ ]:
outputlab3 <- list()
#for (loinc1 in loincs_cofounder_potassium){
#for (loinc1 in bp_cofounder_potassium){
for (loinc1 in c(loincs_cofounder_potassium, bp_cofounder_potassium)){
    for (loinc2 in loincs_potassium){
        key <- paste0(loinc1,loinc2)
        path1 <- paste0(datafolder,'outputlab',loinc1,loinc2,'.parquet')
        outputlab3[[key]] <- arrow::read_parquet(path1)
    }
}

In [ ]:
#for (loinc1 in loincs_cofounder_potassium){
#for (loinc1 in bp_cofounder_potassium){    
for (loinc1 in c(loincs_cofounder_potassium, bp_cofounder_potassium)){
    for (loinc2 in loincs_potassium){
        key <- paste0(loinc1,loinc2)
        print(paste(loinc1, loinc2))        
        pGrunfeld_unbal <- pdata.frame(outputlab3[[key]], index = c("ID", "DAYS_SINCE_ADMIT"))        
        tryCatch({
            print(purtest(pGrunfeld_unbal$res, pmax=2, test ="madwu"))
        }, error=function(e){})
        tryCatch({
            print(cipstest(pGrunfeld_unbal$res, pmax=1))
        }, error=function(e){})
        
    }
}

In [ ]:
# pGrunfeld_unbal <- pdata.frame(outputlab3, index = c("ID", "DAYS_SINCE_ADMIT"))
# print(purtest(pGrunfeld_unbal$res, pmax=3, test ="madwu"))

In [ ]:
pathp <- paste0(datafolder,'outputlab_Pearson.parquet')
pearson <- arrow::read_parquet(pathp)

#testdf <- pearson %>% filter(loinc1==loinc1) %>% filter(loinc2==loinc2)

avg_pearson <- function(n, r){
#    n <- testdf$n
#    r <- testdf$pearson
    z <- fisherz(r)
    k<-length(testdf$ID)
    fisherz2r(sum((n-1)*r)/(sum(n)-3*k))
}


pearson %>% group_by(loinc1, loinc2) %>% summarise(avg_pear=avg_pearson(n,pearson)) %>% ungroup()